# Panle Regression

### Random Effects Panel Regression

In [38]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [39]:
df = pd.read_csv("Dataframes/accuracy_analysis_dataframe.csv")
df.describe()


,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume
count,19441.000000,19432.000000,1.943000e+04,19432.000000,1.952200e+04,1.941100e+04,1.951900e+04,20081.000000,18648.000000,18422.000000,13417.000000,1.970500e+04,18645.000000,18645.000000,19523.000000,19512.000000,19523.000000,2.008000e+04
mean,1.360883,1.267604,1.963704e+02,-3.162201,5.031192e+10,5.915311e+09,6.051035e+10,17.021712,2.325102,2.327782,2.013632,5.763155e+08,123.316668,11.789714,107.145656,4.861111,4.054237,1.099946e+06
std,2.976118,2.828534,2.369072e+04,1197.875143,1.204234e+11,1.170952e+10,1.269863e+11,7.817116,0.376320,0.375747,0.768273,1.717554e+09,213.944612,25.149813,182.876019,10.471280,15.333995,3.178078e+06
min,-16.430000,-15.985000,-8.858503e+03,-159947.826087,5.515891e+07,-9.010000e+09,-8.003695e+10,0.000000,1.000000,1.000000,-0.344403,-4.969700e+10,1.580000,0.000000,0.865000,-22.252345,-83.536836,0.000000e+00
25%,0.496500,0.458180,5.022500e-01,1.865979,1.105947e+10,1.002550e+09,1.374130e+10,12.000000,2.047620,2.050000,1.515266,8.736600e+07,47.666670,4.066060,40.830000,1.812317,-4.024076,0.000000e+00
50%,0.910000,0.859490,4.586500e+00,3.281493,2.038968e+10,2.353000e+09,2.625071e+10,17.000000,2.294120,2.296300,1.999952,2.170000e+08,78.722220,7.004410,68.850000,3.266689,3.929181,4.187605e+05
75%,1.580000,1.485272,1.232875e+01,6.653440,4.298749e+10,5.132000e+09,5.343006e+10,22.000000,2.571430,2.576920,2.413881,5.520000e+08,136.037040,12.488220,118.880000,5.719921,11.962005,1.073648e+06
max,123.650000,126.765710,3.297926e+06,16471.428571,2.892120e+12,1.528710e+11,2.951005e+12,49.000000,3.692310,4.000000,3.754446,3.989800e+10,5950.000000,1178.120860,5908.870000,825.404875,232.448133,1.752907e+08


In [40]:
#removing outliers
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_clean = df.copy()
df_clean["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_clean = df_clean.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_clean = df_clean.dropna()
df_clean.describe()

,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume
count,18655.000000,18655.000000,18655.000000,18655.000000,1.862800e+04,1.861900e+04,1.862800e+04,18655.000000,17915.000000,17708.000000,12860.000000,1.863700e+04,17915.000000,17915.000000,18634.000000,18622.000000,18634.000000,1.865500e+04
mean,1.402044,1.312585,7.004079,5.096519,5.127890e+10,6.010146e+09,6.150797e+10,17.686840,2.324842,2.327326,2.025380,6.067380e+08,125.033309,11.895558,109.048552,4.788334,3.992796,1.111966e+06
std,3.017312,2.876566,17.123315,16.273581,1.220780e+11,1.184507e+10,1.286372e+11,7.243533,0.376329,0.375714,0.761991,1.742313e+09,216.769226,25.476744,185.770335,10.575461,14.843959,3.105971e+06
min,-16.430000,-15.985000,-82.249000,-623.783180,5.515891e+07,-9.010000e+09,-8.003695e+10,1.000000,1.000000,1.000000,-0.344403,-4.969700e+10,1.580000,0.000000,0.865000,-22.252345,-83.536836,0.000000e+00
25%,0.530000,0.498140,0.526500,1.867244,1.132177e+10,1.037600e+09,1.391753e+10,13.000000,2.047620,2.050000,1.515266,9.489400e+07,48.524500,4.089130,41.737241,1.814603,-3.975763,0.000000e+00
50%,0.950000,0.891590,4.412000,3.212207,2.085122e+10,2.406000e+09,2.668716e+10,17.000000,2.292680,2.294120,1.999952,2.274030e+08,79.678570,7.045570,69.900000,3.266878,3.906550,4.345610e+05
75%,1.610000,1.520035,11.532000,6.229381,4.427411e+10,5.221000e+09,5.482497e+10,22.000000,2.571430,2.571430,2.431507,5.750000e+08,137.427275,12.613100,120.505000,5.639640,11.851991,1.089788e+06
max,123.650000,126.765710,95.084000,550.096899,2.892120e+12,1.528710e+11,2.951005e+12,49.000000,3.692310,4.000000,3.754446,3.989800e+10,5950.000000,1178.120860,5908.870000,825.404875,232.448133,1.752907e+08


In [50]:
#df_clean["Date"] = pd.to_datetime(df_clean["Date"])
df_clean

,,GICS Industry Group Name,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume
Date,Instrument,,,,,,,,,,,,,,,,,,,
2022-10-20 07:00:00,POOL.OQ,Retailing,4.78,4.58750,4.196,3.843706,1.259813e+10,1.615339e+09,1.406160e+10,10.0,2.27273,2.200,3.747421,189965000.0,360.11111,24.95082,318.21,2.383926,-9.164375,238539.0
2022-07-21 07:00:00,POOL.OQ,Retailing,7.63,7.51700,1.503,3.625249,1.405795e+10,2.055818e+09,1.556187e+10,9.0,2.20000,2.000,3.092855,307205000.0,435.37500,57.57373,351.23,2.749481,-21.661570,185584.0
2022-04-21 07:00:00,POOL.OQ,Retailing,4.23,3.14867,34.342,6.630736,1.696712e+10,1.412650e+09,1.843682e+10,9.0,2.00000,2.300,2.325202,179203000.0,519.88889,37.49206,422.85,3.422880,-25.160664,290897.0
2022-02-17 07:00:00,POOL.OQ,Retailing,2.63,1.87500,40.267,3.269867,2.268979e+10,1.035557e+09,2.384882e+10,8.0,2.30000,2.300,1.515266,107542000.0,571.00000,41.33833,566.00,4.504382,30.489705,82349.0
2021-10-21 07:00:00,POOL.OQ,Retailing,4.51,3.84833,17.194,2.836295,1.742020e+10,1.411448e+09,1.769954e+10,8.0,2.30000,2.300,1.527139,184573000.0,545.28571,50.48823,434.41,3.684400,-5.130322,157740.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-01-31 08:30:00,AVY.N,Materials,0.69,0.68000,1.471,3.204412,4.877168e+09,1.583900e+09,5.553068e+09,8.0,2.33333,2.625,NaN,42100000.0,54.62500,3.42555,50.19,0.920659,16.003852,157761.0
2013-10-25 08:30:00,AVY.N,Materials,0.69,0.63833,8.095,2.775994,4.273414e+09,1.504900e+09,5.029514e+09,7.0,2.62500,2.875,NaN,62000000.0,47.71429,3.45230,43.52,0.903176,2.467620,236291.0
2013-07-23 08:30:00,AVY.N,Materials,0.71,0.70250,1.068,1.551601,4.258229e+09,1.552300e+09,5.436229e+09,8.0,2.87500,2.875,NaN,70800000.0,45.00000,4.30946,42.76,0.906078,-0.054218,0.0


In [43]:
# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(["Date", "Instrument"], inplace=True)
df_clean.index.set_levels(pd.to_datetime(df.index.levels[1]), level=1, inplace=True)


# define the dependent variable
y = df_clean["Earnings Per Share - Actual Surprise"]

# define the independent variables
X = df_clean[['Market Capitalization', 'Number of Analysts', "Recommendation - Mean (1-5).1", "Net Income after Tax", "3 Month Total Return", "Volume"]]

# perform the random effects panel regression
random_effects_model = RandomEffects(y, X)

# fit the model and print the summary statistics
random_effects_results = random_effects_model.fit()
print(random_effects_results.summary)


KeyError: "None of ['Date', 'Instrument'] are in the columns"